The purpose of this notebook is to explore some data from **Folger Deep**.  The steps in this notebook are:
- Look at the structure of the scalar data file (CM and ABC data)

- Look at Upwelling At Folger Deep, use 24 hour average ABC and CM
  - Load the ABC and CM data
  - Load some prepared upwelling data from a co-located ADCP
  - Plot the two time-series for Two Years (2018-Jan-01 to 2020-Jan-01) to help identify some interesting data

- Look at the structure of the MVBS complex data file
- Load and Plot the MVBS data for two months (June 5th to July 15th, 2019)
- Load the CM data for the functional groups, and plot it over top of the data (June 25th to July 5th, 2019)
- Download the Original Sv data from Folger Deep using the ONC Client Library, and see how it compares to the MVBS data



In [ ]:
#Define the relative paths of the contents of this repo

from pathlib import Path
MVBS_folder = Path.cwd() / 'MVBS_examples'
scalar_folder = Path.cwd() / 'CM_ABC_averaged_examples'
ADCP_folder = Path.cwd() / 'extra_data'

scalar_filename = scalar_folder / "FolgerDeep_20140508_20251230_averaged.mat"

### Step 1:
Look at the structure of the ABC and CM time-series file

In [ ]:
import h5py

def print_mat_structure(filename, root="data_avg"):
    """
    Print structure of a MATLAB v7.3 MAT file in a MATLAB-like format.
    """

    def _print_item(name, obj, indent=0):
        pad = "  " * indent

        if isinstance(obj, h5py.Group):
            print(f"{pad}{name.split('/')[-1]}")
            for key in obj.keys():
                _print_item(f"{name}/{key}", obj[key], indent + 1)

        elif isinstance(obj, h5py.Dataset):
            shape = obj.shape
            dtype = obj.dtype
            print(f"{pad}{name.split('/')[-1]} : {dtype} {shape}")

    with h5py.File(filename, "r") as f:
        if root not in f:
            raise ValueError(f"Root '{root}' not found in file")

        _print_item(root, f[root], indent=0)


# Example usage
if __name__ == "__main__":
    print_mat_structure(scalar_filename)


### Step 2:
Look at Upwelling At Folger Deep, use 24 hour average ABC and CM
- Load the ABC and CM data
- Load some prepared upwelling data from a co-located ADCP
- Plot the two time-series to help identify some interesting data

In [ ]:
#Load the CM, and ABC for Folger Deep
# Use the "pre-averaged" data to make cleaner plots (use 24 hr average instead of 5 minute average)
import h5py
import numpy as np
import pandas as pd

#To convert MATLAB datenum to python datetime
from datetime import datetime, timedelta

#Look at data from Folger Deep
filename = scalar_filename

with h5py.File(filename,'r') as f:
    #Import Daily averaged data
    data = f['data_avg']['hr_24']

    #Load time variable and convert to datetime
    time_MATLAB = np.array(data['time']).squeeze()    
    # The offset for the Unix epoch (1970-01-01) in MATLAB datenum format is 719529
    unix_epoch_offset_matlab = 719529
    time = pd.to_datetime(time_MATLAB - unix_epoch_offset_matlab, unit='D')

    #Get the Time-series data for the 38 kHz channel
    ch_38 = data['ch_38kHz']
    #ch_38 = data['ch_38kHz_fish']
    CM_38 = np.array(ch_38['CM'])
    ABC_38 = np.array(ch_38['ABC'])

    #Get the Time-series data for the 120 kHz channel
    ch_120 = data['ch_120kHz']
    #ch_120 = data['ch_120kHz_krill2']
    CM_120 = np.array(ch_120['CM'])
    ABC_120 = np.array(ch_120['ABC'])



In [ ]:
#Load vertical displacement from the co-located ADCP
#
# This is derived by taking the cumulative sum of the vertical speed at each given depth bin
# This does not represent the total vertical flow over the entire water column

import pandas as pd
import csv
from itertools import islice

def load_csv_pandas(path):
    # skip first 3 lines, use the 4th line as header
    df = pd.read_csv(path, skiprows=3)
    return df  # or: return df.to_dict(orient='records') for list-of-dicts

#Specify the input file:
csv_filename = 'ADCP300kHz_FolgerDeep_2014_2026_verticalDisplacement.csv'
#csv_filename = 'ADCP300kHz_FolgerDeep_2014_2026_verticalFlowSpeed.csv' #Alternate data is flow in m/day
df = load_csv_pandas(ADCP_folder / csv_filename)


In [ ]:
#If using vertical displacement, do this

# Convert the date and time columns to a single datetime column
verticalDisplacement_time = pd.to_datetime(df[['year','month','day','hour','minute','second']])
verticalDisplacement_midColumn = df['52.9 m']
verticalDisplacement_upperColumn = df['16.9 m']

#To make the signal stand out in the plots more clearly, we can 
# detrend the vertical displacement data by removing the linear trend. 
# This will help to highlight any variations in the data that are not due 
# to a simple linear increase or decrease over time.

import numpy as np

def trend_detrend(series):
    mask = series.notna()
    if isinstance(series.index, pd.DatetimeIndex):
        t = series.index.view('int64')/1e9  # seconds since epoch
    else:
        t = np.arange(len(series))
    coef = np.polyfit(t[mask], series[mask], 1)
    trend_all = np.polyval(coef, t)
    detrended = series - trend_all
    detrended[~mask] = np.nan  # restore NaNs
    return detrended

verticalDisplacement_midColumn_detrend = trend_detrend(verticalDisplacement_midColumn)
verticalDisplacement_upperColumn_detrend = trend_detrend(verticalDisplacement_upperColumn)


#if csv_filename == 'ADCP300kHz_FolgerDeep_2014_2026_verticalFlowSpeed.csv':
#    verticalDisplacement_time = pd.to_datetime(df[['year','month','day','hour','minute','second']])
#    verticalFlow_midColumn = df['52.9 m']
#    verticalFlow_upperColumn = df['16.9 m']

In [ ]:
import matplotlib.pyplot as plt

# Zoom to a date range
start_zoom = datetime(2018, 1, 1)
end_zoom = datetime(2020, 1, 1)
#start_zoom = datetime(2024, 1, 1)
#end_zoom = datetime(2026, 1, 1)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(12, 8), sharex=True)

axes[0].plot(time, ABC_38[1,:])  # Plot the second column of CM_38 against time
axes[0].plot(time, ABC_120[1,:])  # Plot the second column of CM_120 against time
axes[0].set_xlabel('Time')
axes[0].set_ylabel('ABC m^2 / m^-2')
axes[0].set_xlim(start_zoom, end_zoom)
axes[0].set_ylim(0, 4*10**-4)
axes[0].legend(['38 kHz', '120 kHz'])
axes[0].grid(True)

#Also, plot some red vertical lines to highlight where upwelling appears to result in a bloom
line_dates = [datetime(2019, 6, 5), datetime(2019, 7, 15)]
# line_dates = [datetime(2025, 9, 5), datetime(2025, 10, 15)]
for line_date in line_dates:
    axes[0].plot( [line_date, line_date], [0, 1],'r--')  
    

if csv_filename == 'ADCP300kHz_FolgerDeep_2014_2026_verticalFlowSpeed.csv':
    axes[1].plot(verticalDisplacement_time, verticalFlow_midColumn)  
    axes[1].set_ylabel('Detrended Vertical \n Flow (m / day)')
    #axes[1].set_ylim(-0.5e7, 1e7)   

elif csv_filename == 'ADCP300kH_FolgerDeep_2014_2026_verticalDisplacement.csv':
    axes[1].plot(verticalDisplacement_time, verticalDisplacement_midColumn_detrend)  
    # axes[1].plot(verticalDisplacement_time, verticalDisplacement_upperColumn_detrend)  
    #axes[2].plot(verticalDisplacement_time, verticalDisplacement_midColumn)  
    axes[1].set_ylabel('Detrended Vertical \n Displacement(m)')
    axes[1].set_ylim(-1.1e7, -0.2e7)   # 2018-2019
    #axes[1].set_ylim(-0.5e7, 1e7)  # 2024-2025

axes[1].set_xlabel('Time')          
axes[1].set_xlim(start_zoom, end_zoom)
# axes[1].legend(['mid-column (52.9 m)', 'upper-column (16.9 m)'])
axes[1].grid(True)



plt.show()

### Step 3:
Look at the structure of the MVBS data

In [ ]:
#Get the filenames of the corresponding MVBS data
MVBS_filename = []
MVBS_filename.append(MVBS_folder / "FolgerDeep_BIOSONICSDTXU08003_20190601T000000_20190701T000000_MVBS_300s_1m.mat")
MVBS_filename.append(MVBS_folder / "FolgerDeep_BIOSONICSDTXU08003_20190701T000000_20190801T000000_MVBS_300s_1m.mat")


In [ ]:
import h5py

def print_mat_structure(filename, root="data"):
    """
    Print structure of a MATLAB v7.3 MAT file in a MATLAB-like format.
    """

    def _print_item(name, obj, indent=0):
        pad = "  " * indent

        if isinstance(obj, h5py.Group):
            print(f"{pad}{name.split('/')[-1]}")
            for key in obj.keys():
                _print_item(f"{name}/{key}", obj[key], indent + 1)

        elif isinstance(obj, h5py.Dataset):
            shape = obj.shape
            dtype = obj.dtype
            print(f"{pad}{name.split('/')[-1]} : {dtype} {shape}")

    with h5py.File(filename, "r") as f:
        if root not in f:
            raise ValueError(f"Root '{root}' not found in file")

        _print_item(root, f[root], indent=0)


# Example usage
if __name__ == "__main__":
    print_mat_structure(MVBS_filename[0])


### Step 4
Load and Plot the MVBS data for 2019-Jun-01 to 2019-Aug-01

In [ ]:
import h5py
import numpy as np
import pandas as pd

#To convert MATLAB datenum to python datetime
from datetime import datetime, timedelta

#Create keys and a dict for the MVBS
channels = ['ch_38kHz','ch_123kHz','ch_210kHz','ch_38kHz_fish','ch_123kHz_krill','ch_123kHz_krill2'] 
MVBS = {k: [] for k in channels}
MVBS_list = {k: [] for k in channels}

#Initialize lists for loading the data from multiple files
time_MATLAB_list = [] #Time variable


for filename in MVBS_filename:
    with h5py.File(filename,'r') as f:
        #Import hourly averaged data
        data = f['data']

        #Load time variable and convert to datetime
        time_MATLAB_list.append(np.array(data['time']).squeeze())    

        #Get the Time-series data for the 38 kHz channel
        MVBS_load = data['MVBS']
        for ch in channels:
            MVBS_list[ch].append(np.array(MVBS_load[ch]))

        #Load the bin depth data. This will be consistent between files
        depth_data = np.array(data['depth_bins'])


#After loading from all files, concatenate the arrays
for ch in channels:
    MVBS[ch] = np.concatenate(MVBS_list[ch], axis=1)

#Also, concatenate the time values, and convert to datetime
# The offset for the Unix epoch (1970-01-01) in MATLAB datenum format is 719529
unix_epoch_offset_matlab = 719529
time_MATLAB = np.concatenate(time_MATLAB_list)
time = pd.to_datetime(time_MATLAB - unix_epoch_offset_matlab, unit='D')

#Delete the list variables to free up memory
del MVBS_list, data


In [ ]:
import matplotlib.pyplot as plt

# 1) Plot MVBS for the 3 channels
# 2) Plot the CM on top of the MVBS data

# Zoom to a date range
start_zoom = datetime(2019, 6, 5)
end_zoom = datetime(2019, 7, 15)

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12, 10), sharex=True)

# Set background color for all subplots
for ax in axes:
    ax.set_facecolor([0.0, 0.0, 0.2])  # Dark Blue

#Plot each channel 
im_list = []
for ii, ch in enumerate(channels[0:3]):
    im = axes[ii].imshow(MVBS[ch],
                   extent = [time.min(), time.max(), depth_data.max(), depth_data.min()],
                   aspect = 'auto',
                   origin = 'lower',
                   vmin=-70,
                   vmax=-45,
                   cmap='turbo')
    im_list.append(im)  # keep reference for colorbar
    
    axes[ii].set_title(ch)
    axes[ii].set_xlabel('Time')
    axes[ii].set_ylabel('Depth [m]')
    axes[ii].set_ylim(90, 0)
    #Zoom to the days when the bloom in productivity appears to happen from the ABC data
    axes[ii].set_xlim(start_zoom, end_zoom)
    

# Add a single colorbar for all subplots
fig.colorbar(im_list[0], ax=axes, orientation='vertical', fraction=0.05, pad=0.04)

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# 1) Plot MVBS for the 3 channels
# 2) Plot the CM on top of the MVBS data

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12, 10), sharex=True)

# Set background color for all subplots
for ax in axes:
    ax.set_facecolor([0.0, 0.0, 0.2])  # Dark Blue

#Plot each channel 
im_list = []
for ii, ch in enumerate(channels[3:]):
    im = axes[ii].imshow(MVBS[ch],
                   extent = [time.min(), time.max(), depth_data.max(), depth_data.min()],
                   aspect = 'auto',
                   origin = 'lower',
                   vmin=-70,
                   vmax=-45,
                   cmap='turbo')
    im_list.append(im)  # keep reference for colorbar
    
    axes[ii].set_title(ch)
    axes[ii].set_xlabel('Time')
    axes[ii].set_ylabel('Depth [m]')
    axes[ii].set_ylim(90, 0)

    #Zoom to the days when the bloom in productivity appears to happen from the ABC data
    axes[ii].set_xlim(start_zoom, end_zoom)

# Add a single colorbar for all subplots
fig.colorbar(im_list[0], ax=axes, orientation='vertical', fraction=0.05, pad=0.04)

plt.show()

### Step 5
Zoom in closer and plot the CM on top

In [ ]:
#Load the 1 hour averaged CM
filename = scalar_filename

with h5py.File(filename,'r') as f:
    #Import Daily averaged data
    data = f['data_avg']['hr_01']

    #Load time variable and convert to datetime
    time_MATLAB = np.array(data['time']).squeeze()    
    # The offset for the Unix epoch (1970-01-01) in MATLAB datenum format is 719529
    unix_epoch_offset_matlab = 719529
    time_CM = pd.to_datetime(time_MATLAB - unix_epoch_offset_matlab, unit='D')

    # Get the CM for the functional groups
    CM_38_fish = np.array(data['ch_38kHz_fish']['CM'])
    CM_123_krill = np.array(data['ch_120kHz_krill']['CM'])
    CM_123_krill2 = np.array(data['ch_120kHz_krill2']['CM'])

In [ ]:
#Plot the MVBS and 1 hour averaged CM

import matplotlib.pyplot as plt

# 1) Plot MVBS for the 3 channels
# 2) Plot the CM on top of the MVBS data

# Zoom to a date range
start_zoom = datetime(2019, 6, 25)
end_zoom = datetime(2019, 7, 5)

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12, 10), sharex=True)

# Set background color for all subplots
for ax in axes:
    ax.set_facecolor([0.0, 0.0, 0.2])  # Dark Blue

#Plot each channel 
im_list = []
for ii, ch in enumerate(channels[3:6]):
    im = axes[ii].imshow(MVBS[ch],
                   extent = [time.min(), time.max(), depth_data.max(), depth_data.min()],
                   aspect = 'auto',
                   origin = 'lower',
                   vmin=-70,
                   vmax=-45,
                   cmap='turbo')
    im_list.append(im)  # keep reference for colorbar
    
    axes[ii].set_title(ch)
    axes[ii].set_xlabel('Time')
    axes[ii].set_ylabel('Depth [m]')
    axes[ii].set_ylim(90, 0)
    #Zoom to the days when the bloom in productivity appears to happen from the ABC data
    axes[ii].set_xlim(start_zoom, end_zoom)
    
    #Plot the CM on top
    if ii == 0:
        axes[ii].plot(time_CM,CM_38_fish[0],'r--')
    elif ii == 1:
        axes[ii].plot(time_CM,CM_123_krill[0],'r--')
    elif ii == 2:
        axes[ii].plot(time_CM,CM_123_krill2[0],'r--')
    


# Add a single colorbar for all subplots
fig.colorbar(im_list[0], ax=axes, orientation='vertical', fraction=0.05, pad=0.04)

plt.show()

### Optional Step - Step 6: 
Create an Oceans3.0 account, and download a few hours of Sv data

In [ ]:
#Request data from Oceans 3.0

from onc.onc import ONC # This requires the ONC API Client Library package

# You will require an ONC account and an ONC token to proceed
# To register, visit: https://data.oceannetworks.ca/Registration
#
# Once registered:
# 1) access your Profile from the top-right of https://data.oceannetworks.ca/
# 2) Click the "Web Services API" tab
# 3) Copy your token
ONC_token = '12345678-abcdefgh-9012345-ijklmno' # Add your token here


In [ ]:
#NOTE: These files are very large, so just 2 hours of data can take a while to download

#Initialize the ONC object for retrieving data
onc = ONC(ONC_token)

locationCode = 'FGPD'
deviceCategoryCode = 'ECHOSOUNDERBIOA'
deviceCode = 'BIOSONICSDTXU08003'
start_time = '2019-06-28T00:00:00.000Z'
end_time = '2019-06-28T02:00:00.000Z'
ensemblePeriod = 0

#Download in MAT :
dataProductCode = 'BSTS' # 'BioSonics Time Series'
filters = {
    'locationCode':locationCode,
    'deviceCategoryCode':deviceCategoryCode,
    'dataProductCode':dataProductCode,
    'extension': 'mat',
    #'deviceCode':deviceCode,
    'dateFrom': start_time,
    'dateTo'  : end_time,
    'dpo_ensemblePeriod': ensemblePeriod, # 0 = data not altered (none). All others in s: 20 (30s), 60, 300, 600, 900, 3600
    'dpo_calibration': 1, # 1 = Sv. All others: 2=Ts, 0=rawCounts
    'dpo_rangeAveraging':0, # 0=unaveraged, All others: 0.05, 0.1, 0.25, 0.5, 1
    'token': ONC_token
    }

result = onc.requestDataProduct(filters)
runData = onc.runDataProduct(result['dpRequestId'], waitComplete=True)
print(runData)
result = onc.downloadDataProduct(runData['runIds'][0])

In [ ]:
#This will have saved to a subfolder called "output"

from pathlib import Path

#Print the names of the downloaded files
Sv_folder = Path("output")  

for f in Sv_folder.glob("*.mat"):
    print(f.name)

In [ ]:
#Load and plot the data
# Plot the Sv left, and MVBS right
from scipy.io import loadmat
import numpy as np
import pandas as pd

#Get the list of all mat-files in the data-directory
mat_path = list(Sv_folder.glob('*.mat'))


#Initialize some variables
frequency = np.zeros(3)
channels_Sv = []
depth_bins = []

#Go through all MAT files and load
for mat_file in mat_path:
    mat_dict = loadmat(mat_file,simplify_cells=True)
    #print(mat_dict.keys())
    data = mat_dict['data']
    meta = mat_dict['meta']

    #Get list info, and create keys
    if frequency[0] == 0:
        for ch in range(len(data)):
            frequency[ch] = data[ch]['frequency']/1000
            channels_Sv.append("ch_{}kHz".format(int(frequency[ch])))
            depth_bins.append(np.array(meta['depth'] - data[ch]['range']))

        Sv = {k: [] for k in channels_Sv}
        Sv_list = {k: [] for k in channels_Sv}
        time_MATLAB = {k: [] for k in channels_Sv}
        time_MATLAB_list = {k: [] for k in channels_Sv}
        time_Sv = {k: [] for k in channels_Sv}

    #Step through each channel in the data    
    for ii,ch in enumerate(channels_Sv):
        time_MATLAB_list[ch].append(np.array(data[ii]['time']))
        Sv_list[ch].append(np.array(data[ii]['vals']))
        #print(np.array(data[ii]['vals']).shape)


#After loading from all files, concatenate the arrays
for ch in channels_Sv:
    Sv[ch] = np.concatenate(Sv_list[ch], axis=1)

    #Also, concatenate the time values, and convert to datetime
    # The offset for the Unix epoch (1970-01-01) in MATLAB datenum format is 719529
    unix_epoch_offset_matlab = 719529
    time_MATLAB[ch] = np.concatenate(time_MATLAB_list[ch])
    time_Sv[ch] = pd.to_datetime(time_MATLAB[ch] - unix_epoch_offset_matlab, unit='D')

#Delete the list variables to free up memory
del Sv_list, data, time_MATLAB, time_MATLAB_list





In [ ]:
#Plot the MVBS and Sv side by side

import matplotlib.pyplot as plt

from datetime import datetime, timedelta

# 1) Plot MVBS for the 3 channels
# 2) Plot Sv for the 3 channels

# Zoom to a date range
start_zoom = datetime(2019, 6, 28)
end_zoom = datetime(2019, 6, 28, 2, 0, 0)

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 10), sharex=True)

# Set background color for all subplots
for ax in axes.flatten():
    ax.set_facecolor([0.0, 0.0, 0.2])  # Dark Blue

#Plot each channel of the MVBS data
im_list = []
for jj in range(2):
    for ii, ch in enumerate(channels[0:3]):
        if jj==0:
            #Plot the MVBS data
            im = axes[ii,0].imshow(MVBS[ch],
                    extent = [time.min(), time.max(), depth_data.max(), depth_data.min()],
                    aspect = 'auto',
                    origin = 'lower',
                    vmin=-70,
                    vmax=-45,
                    cmap='turbo')
            
        elif jj==1:
             #Plot the Sv data
             im = axes[ii,1].imshow(Sv[ch],
                   extent = [time_Sv[ch].min(), time_Sv[ch].max(), depth_bins[ii].max(), depth_bins[ii].min()],
                   aspect = 'auto',
                   origin = 'lower',
                   vmin=-70,
                   vmax=-45,
                   cmap='turbo')

        im_list.append(im)  # keep reference for colorbar
        
        axes[ii,jj].set_title(ch)
        axes[ii,jj].set_xlabel('Time')
        axes[ii,jj].set_ylabel('Depth [m]')
        axes[ii,jj].set_ylim(90, 0)
        #Zoom to the days when the bloom in productivity appears to happen from the ABC data
        axes[ii,jj].set_xlim(start_zoom, end_zoom)
    

# Add a single colorbar for all subplots
fig.colorbar(im_list[0], ax=axes, orientation='vertical', fraction=0.05, pad=0.04)

plt.show()

### Observations

- The original Sv data is very rich in detail that is lost with the averaged MVBS data
- However, the MVBS data can be use to tell us where to look in the Sv data for interesting features and events